In [1]:
# SageMaker 노트북에서 실행:
import os
from pathlib import Path

# 현재 위치
print("Current dir:", os.getcwd())

# project_release 찾기
home = Path.home()
print("\nSearching for project_release...")
for path in home.rglob("project_release"):
    print(f"Found: {path}")
    
# train_corpus.txt 찾기
print("\nSearching for train_corpus.txt...")
for path in home.rglob("*corpus.txt"):
    print(f"Found: {path}")

Current dir: /home/sagemaker-user/project_release

Searching for project_release...
Found: /home/sagemaker-user/project_release

Searching for train_corpus.txt...
Found: /home/sagemaker-user/project_release/Amazon_products/test/test_corpus.txt
Found: /home/sagemaker-user/project_release/Amazon_products/train/train_corpus.txt


In [2]:
%run silver_label_generation_sagemaker.py

SILVER LABEL GENERATION

[1/5] Loading metadata...
  ✓ Loaded 531 classes
  ✓ Loaded 531 keyword sets
  ✓ Loaded 525 hierarchy edges

[2/5] Loading training corpus...
  ✓ Loaded 29487 training samples

[3/5] Generating silver labels...
  • TOP_K = 3
  • MIN_KEYWORD_MATCHES = 1


Generating silver labels: 100%|██████████| 29487/29487 [01:05<00:00, 447.58it/s]



[4/5] Saving results...
  ✓ Silver labels saved to: /home/sagemaker-user/project_release/Amazon_products/outputs/silver_labels.json
  ✓ Statistics saved to: /home/sagemaker-user/project_release/Amazon_products/outputs/silver_labels_stats.json

[5/5] Summary
Total samples:        29,487
Labeled samples:      25,920
Discarded samples:    3,567
Avg labels/sample:    5.96

Top 10 most frequent labels:
------------------------------------------------------------
    0  grocery_gourmet_food                 15,330
   40  baby_products                        11,231
    3  toys_games                            6,951
  169  pet_supplies                          6,205
  265  candy_chocolate                       5,251
  145  bathing_skin_care                     4,973
  213  breakfast_foods                       4,650
  214  cereals                               4,268
   21  baby_toddler_toys                     4,036
   44  skin_care                             3,889

✅ DONE!


In [4]:
%run tfidf_baseline_fast.py

TF-IDF BASELINE MODEL (FAST)

[1/6] Loading corpus...
  ✓ Train: 29,487 samples
  ✓ Test: 19,658 samples

[2/6] Loading silver labels...
  ✓ Loaded silver labels for 25,920 samples

[3/6] Loading hierarchy...
  ✓ Loaded 525 hierarchy edges

  → Using 25,920 / 29,487 labeled samples

[4/6] Creating TF-IDF vectors...
  • Max features: 10,000
  → Fitting on all train texts...
  → Transforming test texts...

  → Filtering classes...
  • Valid classes: 381 / 531
  • Removed classes: 150 (< 2 samples)

  ✓ X_train: (25920, 10000)
  ✓ y_train: (25920, 381)
  ✓ X_test: (19658, 10000)

[5/6] Training model...

[Training] Training classifier...
  • Input dim: 10000
  • Num classes: 381
  • Training samples: 25920
  → Training... (this may take 5-10 minutes)


/opt/conda/envs/esci/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


  ✓ Training complete!

[6/6] Predicting on test set...

[Prediction] Predicting top-3 labels...
  → Computing scores...
  → Selecting top-k...


Post-processing: 100%|██████████| 19658/19658 [00:00<00:00, 39422.32it/s]


  ✓ Predictions complete!

[7/7] Generating submission file...
  ✓ Submission saved to: /home/sagemaker-user/project_release/Amazon_products/outputs/submission_tfidf_baseline.csv

SUMMARY
Training samples:     25,920
Test samples:         19,658
Vocabulary size:      10,000
Avg labels per pred:  3.11

Submission preview:
  id       label
0  0    0,40,169
1  1     3,17,40
2  2    0,40,169
3  3     0,1,216
4  4   40,44,145
5  5     0,8,206
6  6    0,40,313
7  7  40,100,145
8  8    23,24,49
9  9     3,21,40

✅ DONE! Ready for Kaggle submission


In [5]:
%run _selftrain_baseline.py

SELF-TRAINING WITH PSEUDO-LABELING

[1/9] Loading corpus...
  ✓ Train: 29,487 samples
  ✓ Test: 19,658 samples

[2/9] Loading silver labels...
  ✓ Loaded 25,920 silver labels

[3/9] Loading hierarchy...
  ✓ Loaded 525 hierarchy edges

  → Labeled: 25,920
  → Unlabeled: 3,567

[4/9] Creating TF-IDF vectors...
  ✓ Valid classes: 381 / 531

[5/9] Training initial model (silver labels only)...

[Training] Training classifier...
  • Input dim: 10000
  • Num classes: 381
  • Training samples: 25920
  → Training...


/opt/conda/envs/esci/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


  ✓ Training complete!

[6/9] Generating pseudo-labels...

[Pseudo-labeling] Generating pseudo-labels...
  • Confidence threshold: 0.5
  • Top-K for pseudo: 1
  • Hierarchy depth: 1
  • Unlabeled samples: 3,567
  → Computing scores...
  → Selecting high-confidence predictions...


Processing: 100%|██████████| 3567/3567 [00:00<00:00, 52701.02it/s]


  ✓ Pseudo-labels generated!
    • Accepted: 2,383 / 3,567 (66.8%)
    • Avg confidence: 0.600
    • Avg labels/sample: 1.00
  ✓ Pseudo-labels saved to: /home/sagemaker-user/project_release/Amazon_products/outputs/pseudo_labels.json

[7/9] Combining silver + pseudo labels...


  ✓ Combined training set: 28,303 samples
    • Silver labels: 25,920
    • Pseudo labels: 2,383

[8/9] Retraining with combined data...

[Training] Training classifier...
  • Input dim: 10000
  • Num classes: 381
  • Training samples: 28303
  → Training...


/opt/conda/envs/esci/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/envs/esci/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

  ✓ Training complete!

[9/9] Predicting on test set...

[Prediction] Predicting top-3 labels...
  → Computing scores...
  → Selecting top-k...


Post-processing: 100%|██████████| 19658/19658 [00:00<00:00, 40056.02it/s]


  ✓ Predictions complete!

[Final] Generating submission file...
  ✓ Submission saved to: /home/sagemaker-user/project_release/Amazon_products/outputs/submission_selftrain.csv

SUMMARY
Initial training:     25,920 samples
Pseudo-labels added:  2,383 samples
Final training:       28,303 samples
Improvement:          +9.2% data

Test predictions:     19,658 samples
Avg labels per pred:  3.11

✅ DONE! Ready for Kaggle submission


In [6]:
%run gnn_baseline__.py

GNN-BASED HIERARCHICAL CLASSIFICATION

[1/8] Loading data...
  ✓ Train: 29,487 samples
  ✓ Test: 19,658 samples

[2/8] Loading metadata...
  ✓ Classes: 531
  ✓ Hierarchy edges: 1136
  ✓ Silver labels: 25,920

[3/8] Building label graph...
  ✓ Adjacency matrix: torch.Size([531, 531])

[4/8] Creating TF-IDF vectors...
  ✓ Valid classes: 381 / 531
  ✓ Training samples: 25,920

[5/8] Initializing GNN model...
  ✓ Model initialized
  ✓ Parameters: 5,267,968

[6/8] Training GNN model...
  Epoch 1/3 - Loss: 0.1724
  Epoch 2/3 - Loss: 0.0950
  Epoch 3/3 - Loss: 0.0799
  ✓ Training complete!

[7/8] Predicting on test set...

[8/8] Generating submission...
  ✓ Submission saved to: /home/sagemaker-user/project_release/Amazon_products/outputs/submission_gnn.csv

SUMMARY
Model type:           GNN (LabelGCN)
GNN layers:           2
Hidden dim:           128
Hierarchy depth:      1
Training samples:     25,920
Test predictions:     19,658
Avg labels per pred:  4.22

✅ DONE! Ready for Kaggle submissio

In [7]:
%run gnn_baseline__2.py

GNN-BASED HIERARCHICAL CLASSIFICATION

[1/8] Loading data...
  ✓ Train: 29,487 samples
  ✓ Test: 19,658 samples

[2/8] Loading metadata...
  ✓ Classes: 531
  ✓ Hierarchy edges: 1136
  ✓ Silver labels: 25,920

[3/8] Building label graph...
  ✓ Adjacency matrix: torch.Size([531, 531])

[4/8] Creating TF-IDF vectors...
  ✓ Valid classes: 421 / 531
  ✓ Training samples: 25,920

[5/8] Initializing GNN model...
  ✓ Model initialized
  ✓ Parameters: 5,273,088

[6/8] Training GNN model...
  Epoch 1/5 - Loss: 0.1606
  Epoch 2/5 - Loss: 0.0870
  Epoch 3/5 - Loss: 0.0653
  Epoch 4/5 - Loss: 0.0609
  Epoch 5/5 - Loss: 0.0590
  ✓ Training complete!

[7/8] Predicting on test set...

[8/8] Generating submission...
  ✓ Submission saved to: /home/sagemaker-user/project_release/Amazon_products/outputs/submission_gnn.csv

SUMMARY
Model type:           GNN (LabelGCN)
GNN layers:           2
Hidden dim:           128
Hierarchy depth:      1
Training samples:     25,920
Test predictions:     19,658
Avg labe

In [9]:
%run gnn_baseline__3.py

GNN-BASED HIERARCHICAL CLASSIFICATION

[1/8] Loading data...
  ✓ Train: 29,487 samples
  ✓ Test: 19,658 samples

[2/8] Loading metadata...
  ✓ Classes: 531
  ✓ Hierarchy edges: 1136
  ✓ Silver labels: 25,920

[3/8] Building label graph...
  ✓ Adjacency matrix: torch.Size([531, 531])

[4/8] Creating TF-IDF vectors...
  ✓ Valid classes: 421 / 531
  ✓ Training samples: 25,920

[5/8] Initializing GNN model...
  ✓ Model initialized
  ✓ Parameters: 5,404,544

[6/8] Training GNN model...
  Epoch 1/20 - Loss: 0.1501
  Epoch 2/20 - Loss: 0.0790
  Epoch 3/20 - Loss: 0.0634
  Epoch 4/20 - Loss: 0.0598
  Epoch 5/20 - Loss: 0.0581
  Epoch 6/20 - Loss: 0.0568
  Epoch 7/20 - Loss: 0.0557
  Epoch 8/20 - Loss: 0.0533
  Epoch 9/20 - Loss: 0.0520
  Epoch 10/20 - Loss: 0.0506
  Epoch 11/20 - Loss: 0.0496
  Epoch 12/20 - Loss: 0.0486
  Epoch 13/20 - Loss: 0.0469
  Epoch 14/20 - Loss: 0.0448
  Epoch 15/20 - Loss: 0.0430
  Epoch 16/20 - Loss: 0.0411
  Epoch 17/20 - Loss: 0.0392
  Epoch 18/20 - Loss: 0.0378
 

In [11]:
%run case_study_extraction__.py

CASE STUDY EXTRACTION

[1/5] Loading data...
  ✓ Test corpus: 19,658 samples
  ✓ Classes: 531

[2/5] Loading predictions...
  ✓ TF-IDF predictions: 19,658
  ✓ GNN predictions: 19,658

[3/5] Finding success cases...
  ✓ Found 5 success examples

[4/5] Finding failure cases...
  ✓ Found 5 failure examples

[5/5] Saving results...
  ✓ Case study saved to: /home/sagemaker-user/project_release/Amazon_products/outputs/case_study_examples.json

SUCCESS CASE #1 (Strong keyword evidence)
Product ID: 79
Text: lolita lempicka l de lolita lempicka eau de parfum spray this is such a feminine scent mysterious amber , woody depths , light floral notes and a touch of crisp citrus . but amazingly , it 's light ! ...

TF-IDF Predicted (4 labels):
  - 10: beauty
  - 23: health_personal_care
  - 24: personal_care

Keyword matches:
  - beauty: fragrance
  - personal_care: fragrance
  - fragrance: scent, parfum

GNN Predicted (3 labels):
  - 10: beauty
  - 23: health_personal_care
  - 220: fragrance
  Overl